In [247]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf

tf.enable_eager_execution()

from matplotlib import font_manager, rc

rc('font', family='AppleGothic') #for mac

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pprint import pprint
import numpy as np
import os

print(tf.__version__)

1.14.0


In [246]:
import pandas as pd
df = pd.read_json("moviedata2.json")[:5000]

In [201]:
import re
df.movie_summary = df.movie_summary.apply(lambda x: str(x))
df.movie_summary = df.movie_summary.apply(lambda x: re.sub("[^A-Za-z. 0-9]", '', x))
df.movie_synopsis = df.movie_synopsis.apply(lambda x : str(x))
df.movie_synopsis = df.movie_synopsis.apply(lambda x : re.sub('[^A-Za-z. 0-9]', '', x))
df.movie_tagline = df.movie_tagline.apply(lambda x: str(x))



In [202]:
tl = list(df["movie_tagline"])
tl = [" ".join([ j for j in eval(i) if j != "\n"]).strip() for i in tl]

In [203]:
df.movie_tagline = tl
len(df)

5000

In [204]:
df.movie_tagline = df.movie_tagline.apply(lambda x: re.sub('[^A-Za-z. 0-9]', '', x))

In [123]:
sources = [['I', 'feel', 'hungry'],
     ['tensorflow', 'is', 'very', 'difficult'],
     ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'],
     ['tensorflow', 'is', 'very', 'fast', 'changing']]
targets = [['나는', '배가', '고프다'],
           ['텐서플로우는', '매우', '어렵다'],
           ['텐서플로우는', '딥러닝을', '위한', '프레임워크이다'],
           ['텐서플로우는', '매우', '빠르게', '변화한다']]

In [205]:
sources[0]

['Four',
 'teen',
 'girls',
 'diving',
 'in',
 'a',
 'ruined',
 'underwater',
 'city',
 'quickly',
 'learn',
 'theyve',
 'entered',
 'the',
 'territory',
 'of',
 'the',
 'deadliest',
 'shark',
 'species',
 'in',
 'the',
 'claustrophobic',
 'labyrinth',
 'of',
 'submerged',
 'caves.',
 '47',
 'Meters',
 'Down',
 'Uncaged',
 'follows',
 'the',
 'diving',
 'adventure',
 'of',
 'four',
 'teenage',
 'girls',
 'Sophie',
 'Nlisse',
 'Corinne',
 'Foxx',
 'Brianne',
 'Tju',
 'and',
 'Sistine',
 'Stallone',
 'exploring',
 'a',
 'submerged',
 'Mayan',
 'City.',
 'Once',
 'inside',
 'their',
 'rush',
 'of',
 'excitement',
 'turns',
 'into',
 'a',
 'jolt',
 'of',
 'terror',
 'as',
 'they',
 'discover',
 'the',
 'sunken',
 'ruins',
 'are',
 'a',
 'hunting',
 'ground',
 'for',
 'deadly',
 'Great',
 'White',
 'Sharks.',
 'With',
 'their',
 'air',
 'supply',
 'steadily',
 'dwindling',
 'the',
 'friends',
 'must',
 'navigate',
 'the',
 'underwater',
 'labyrinth',
 'of',
 'claustrophobic',
 'caves',
 'an

In [77]:
#sources = [i.split() for i in df['movie_summary']]

def lem(summary):

    tokens = word_tokenize(summary)
    porter = PorterStemmer()
    stems = [porter.stem(t) for t in tokens]
    stopwords = nltk.corpus.stopwords.words('english')
    words = [w for w in stems if w.lower() not in stopwords]
    s = list(set(words))
        
    return s
df['movie_summary'] = df['movie_summary'].apply(lem)

In [206]:
sources = [i.split() for i in df['movie_summary']]
targets = [i.split() for i in df['movie_tagline']]

# sources를 stemming해야함 하지만 어떻게?

In [248]:
targets[0]

['The', 'Next', 'Chapter']

In [212]:
source = []
for s in sources:
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(t) for t in s]
    stopwords = nltk.corpus.stopwords.words('english') 
    words = [w for w in lemmas if w.lower() not in stopwords]
    list_ = list(set(words))
    source.append(list_)

In [213]:
print(len(source[0]))

66


In [214]:
r1 = re.compile(r"[^A-Za-z {1,}0-9]")
ts_vocab = [r1.sub("",j).lower() for i in source for j in i ]
# s_vocab = list(set(sum(sources, [])))
s_vocab = list(set(ts_vocab))
s_vocab.sort()
s_vocab.pop(0) #0번째에 있던 공백 없앰

''

In [215]:
len(s_vocab)

41086

In [216]:
source2idx = {word : idx for idx, word in enumerate(s_vocab)}
idx2source = {idx : word for idx, word in enumerate(s_vocab)}

pprint(len(source2idx))


41086


In [244]:
targets

[['The', 'Next', 'Chapter'],
 ['Youll', 'Float', 'Again'],
 ['Its', 'Alien', 'the', '8th', 'passenger.'],
 [],
 ['All',
  'we',
  'have',
  'to',
  'decide',
  'is',
  'what',
  'to',
  'do',
  'with',
  'the',
  'time',
  'that',
  'we',
  'are',
  'given'],
 ['Nobody', 'Runs', 'Forever'],
 ['The', 'Magic', 'Begins', 'November', '16th.'],
 ['Its', 'just', 'a', 'bunch', 'of', 'Hocus', 'Pocus'],
 ['Landing', 'This', 'Christmas.'],
 ['71',
  'of',
  'the',
  'Earths',
  'surface',
  'is',
  'covered',
  'by',
  'water.',
  'Thats',
  'a',
  'lot',
  'of',
  'space',
  'to',
  'find',
  'one',
  'fish.'],
 [],
 ['The', 'dead', 'are', 'alive'],
 ['A',
  'place',
  'beyond',
  'your',
  'dreams.',
  'A',
  'movie',
  'beyond',
  'your',
  'imagination.'],
 [],
 ['The',
  'truth',
  'is',
  'always',
  'in',
  'the',
  'last',
  'place',
  'you',
  'look.'],
 ['Some', 'games', 'you', 'play', 'some', 'you', 'survive.'],
 [],
 ['Cruise.', 'Kidman.', 'Kubrick.'],
 ['You',
  'know',
  'his',
  '

In [236]:
# vocabulary for targets
r1 = re.compile(r"[^A-Za-z {1,}0-9]")
ts_vocab = [r1.sub("",j).lower() for i in targets for j in i ]
t_vocab = list(set(ts_vocab))
t_vocab.sort()
t_vocab = ['<pad>', '<bos>', '<eos>'] + t_vocab
t_vocab = [x for x in t_vocab if x]
print(t_vocab)
target2idx = {word : idx for idx, word in enumerate(t_vocab)}
idx2target = {idx : word for idx, word in enumerate(t_vocab)}


['<pad>', '<bos>', '<eos>', '1', '10', '100', '1000', '10000', '102', '11', '12', '12th', '13', '135th', '13th', '14', '1423', '146', '15', '150', '1500', '16', '1613', '16th', '17', '1755', '1766', '18', '18000', '19', '1921', '1923', '1925', '1926', '1928', '1929', '1930', '1931', '1932', '1933', '1934', '1940', '1941', '1944', '1946', '1950s', '1951', '1952', '1953', '1956', '19561957', '1962', '1963', '1964', '1965', '1968', '1972', '1973', '1975', '1975white', '1976', '1984', '1991', '1992', '1993', '1times', '2', '20', '200', '2000', '20000', '2002', '2005', '2006', '2008', '2013', '2015', '2019', '2019s', '2022', '2024', '20th', '21', '212', '21st', '22', '22nd', '23', '24', '24th', '25', '25th', '26', '26000', '2611', '261108', '27558', '28', '283', '29', '2nd', '3', '30', '300000', '30yearold', '31', '34', '35', '353260000', '3d', '4', '40', '40000', '410000tax', '420', '45', '45caliber', '48', '49', '4track', '5', '50', '500', '5000', '55', '6', '60', '600', '6000', '60000', 

In [243]:
target2idx

{'<pad>': 0,
 '<bos>': 1,
 '<eos>': 2,
 '1': 3,
 '10': 4,
 '100': 5,
 '1000': 6,
 '10000': 7,
 '102': 8,
 '11': 9,
 '12': 10,
 '12th': 11,
 '13': 12,
 '135th': 13,
 '13th': 14,
 '14': 15,
 '1423': 16,
 '146': 17,
 '15': 18,
 '150': 19,
 '1500': 20,
 '16': 21,
 '1613': 22,
 '16th': 23,
 '17': 24,
 '1755': 25,
 '1766': 26,
 '18': 27,
 '18000': 28,
 '19': 29,
 '1921': 30,
 '1923': 31,
 '1925': 32,
 '1926': 33,
 '1928': 34,
 '1929': 35,
 '1930': 36,
 '1931': 37,
 '1932': 38,
 '1933': 39,
 '1934': 40,
 '1940': 41,
 '1941': 42,
 '1944': 43,
 '1946': 44,
 '1950s': 45,
 '1951': 46,
 '1952': 47,
 '1953': 48,
 '1956': 49,
 '19561957': 50,
 '1962': 51,
 '1963': 52,
 '1964': 53,
 '1965': 54,
 '1968': 55,
 '1972': 56,
 '1973': 57,
 '1975': 58,
 '1975white': 59,
 '1976': 60,
 '1984': 61,
 '1991': 62,
 '1992': 63,
 '1993': 64,
 '1times': 65,
 '2': 66,
 '20': 67,
 '200': 68,
 '2000': 69,
 '20000': 70,
 '2002': 71,
 '2005': 72,
 '2006': 73,
 '2008': 74,
 '2013': 75,
 '2015': 76,
 '2019': 77,
 '2019s': 

In [238]:
def preprocess(sequences, max_len, dic, mode = 'source'):
    assert mode in ['source', 'target'], 'source와 target 중에 선택해주세요.'
    
    if mode == 'source':
        # preprocessing for source (encoder)
        s_input = list(map(lambda sentence : [dic.get(token) for token in sentence], sequences))
        s_input = [[x for x in each_input if x] for each_input in s_input]
        s_len = list(map(lambda sentence : len(sentence), s_input))
        s_input = pad_sequences(sequences = s_input, maxlen = max_len, padding = 'post', truncating = 'post')
        return s_len, s_input
    

    elif mode == 'target':
        # preprocessing for target (decoder)
        # input
        t_input = list(map(lambda sentence : ['<bos>'] + sentence + ['<eos>'], sequences))
        t_input = [[x for x in each_input if x] for each_input in t_input]
        t_input = list(map(lambda sentence : [dic.get(token) for token in sentence], t_input))
        t_len = list(map(lambda sentence : len(sentence), t_input))
        t_input = pad_sequences(sequences = t_input, maxlen = max_len, padding = 'post', truncating = 'post')
     
        # output
        t_output = list(map(lambda sentence : sentence + ['<eos>'], sequences))
        t_output = list(map(lambda sentence : [dic.get(token) for token in sentence], t_output))
        t_output = pad_sequences(sequences = t_output, maxlen = max_len, padding = 'post', truncating = 'post')
        
        return t_len, t_input, t_output

In [241]:
# preprocessing for source
s_max_len = 12
s_len, s_input = preprocess(sequences = sources,
                            max_len = s_max_len, dic = source2idx, mode = 'source')
print(s_len, s_input)

[82, 75, 288, 0, 391, 197, 724, 372, 399, 472, 37, 571, 451, 114, 437, 198, 56, 478, 270, 444, 132, 377, 626, 482, 185, 286, 274, 224, 329, 656, 436, 284, 265, 157, 405, 557, 429, 317, 338, 110, 258, 54, 482, 257, 317, 308, 263, 136, 20, 266, 515, 298, 196, 497, 176, 494, 708, 258, 53, 465, 112, 335, 282, 703, 503, 271, 598, 129, 448, 285, 137, 608, 217, 181, 316, 545, 124, 234, 59, 270, 53, 445, 7, 368, 313, 413, 10, 85, 398, 437, 616, 154, 266, 24, 564, 46, 385, 525, 23, 579, 162, 360, 366, 695, 358, 194, 504, 196, 15, 303, 504, 354, 150, 354, 365, 134, 215, 212, 297, 210, 395, 347, 79, 411, 78, 382, 37, 342, 170, 388, 518, 201, 18, 20, 23, 20, 502, 426, 245, 210, 36, 192, 454, 512, 56, 90, 15, 135, 172, 402, 417, 607, 130, 82, 380, 795, 205, 338, 520, 328, 298, 14, 131, 388, 21, 322, 168, 174, 12, 114, 203, 615, 224, 23, 307, 97, 179, 317, 322, 604, 115, 117, 340, 568, 219, 264, 66, 271, 382, 463, 137, 238, 99, 32, 29, 278, 102, 218, 252, 283, 420, 404, 394, 294, 122, 165, 63, 202, 

In [242]:
# preprocessing for target
t_max_len = 12
t_len, t_input, t_output = preprocess(sequences = targets,
                                      max_len = t_max_len, dic = target2idx, mode = 'target')
print(t_len, t_input, t_output)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [31]:
# hyper-parameters
epochs = 100
batch_size = 4
learning_rate = .005
total_step = epochs / batch_size
buffer_size = 100
n_batch = buffer_size//batch_size
embedding_dim = 32
units = 128

# input
data = tf.data.Dataset.from_tensor_slices((s_len, s_input, t_len, t_input, t_output))
data = data.shuffle(buffer_size = buffer_size)
data = data.batch(batch_size = batch_size)
# s_mb_len, s_mb_input, t_mb_len, t_mb_input, t_mb_output = iterator.get_next()

In [32]:
data

<DatasetV1Adapter shapes: ((?,), (?, 10), (?,), (?, 12), (?, 12)), types: (tf.int32, tf.int32, tf.int32, tf.int32, tf.int32)>

In [33]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
    if tf.test.is_gpu_available():
        return tf.keras.layers.CuDNNGRU(units, 
                                        return_sequences=True, 
                                        return_state=True, 
                                        recurrent_initializer='glorot_uniform')
    else:
        return tf.keras.layers.GRU(units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')

In [34]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
#         print("state: {}".format(state.shape))
#         print("output: {}".format(state.shape))
              
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [35]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        # * `score = FC(tanh(FC(EO) + FC(H)))`
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
                
        #* `attention weights = softmax(score, axis = 1)`. Softmax by default is applied on the last axis but here we want to apply it on the *1st axis*, since the shape of score is *(batch_size, max_length, 1)*. `Max_length` is the length of our input. Since we are trying to assign a weight to each input, softmax should be applied on that axis.
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        # * `context vector = sum(attention weights * EO, axis = 1)`. Same reason as above for choosing axis as 1.
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        # * `embedding output` = The input to the decoder X is passed through an embedding layer.
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        # * `merged vector = concat(embedding output, context vector)`
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [36]:
encoder = Encoder(len(source2idx), embedding_dim, units, batch_size)
decoder = Decoder(len(target2idx), embedding_dim, units, batch_size)

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    
#     print("real: {}".format(real))
#     print("pred: {}".format(pred))
#     print("mask: {}".format(mask))
#     print("loss: {}".format(tf.reduce_mean(loss_)))
    
    return tf.reduce_mean(loss_)

# creating optimizer
optimizer = tf.train.AdamOptimizer()

# creating check point (Object-based saving)
checkpoint_dir = './data_out/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                encoder=encoder,
                                decoder=decoder)

# create writer for tensorboard
summary_writer = tf.contrib.summary.create_file_writer(logdir=checkpoint_dir)

W0930 19:47:59.557367 21676 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
EPOCHS = 1

for epoch in range(EPOCHS):
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for i, (s_len, s_input, t_len, t_input, t_output) in enumerate(data):
        loss = 0
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(s_input, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([target2idx['<bos>']] * batch_size, 1)
            
            #Teacher Forcing: feeding the target as the next input
            for t in range(1, t_input.shape[1]):
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(t_input[:, t], predictions)
            
                dec_input = tf.expand_dims(t_input[:, t], 1) #using teacher forcing
                
        batch_loss = (loss / int(t_input.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradient = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradient, variables))
        
    if epoch % 10 == 0:
        #save model every 10 epoch
        print('Epoch {} Loss {:.4f} Batch Loss {:.4f}'.format(epoch,
                                            total_loss / n_batch,
                                            batch_loss.numpy()))
        checkpoint.save(file_prefix = checkpoint_prefix)

In [ ]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
#     sentence = preprocess_sentence(sentence)

    inputs = [inp_lang[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weigths to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += idx2target[predicted_id] + ' '

        if idx2target.get(predicted_id) == '<eos>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

# result, sentence, attention_plot = evaluate(sentence, encoder, decoder, source2idx, target2idx,
#                                             s_max_len, t_max_len)

In [ ]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    
    fontdict = {'fontsize': 14}
    
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    plt.show()

In [ ]:
def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    result, sentence, attention_plot = evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
        
    print('Input: {}'.format(sentence))
    print('Predicted translation: {}'.format(result))
    
    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [ ]:
#restore checkpoint

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
sentence = 'I feel hungry'
# sentence = 'tensorflow is a framework for deep learning'

translate(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)